In [1]:
!pip3 install datasets

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os
import sys

# from torch_geometric.data import Data, DataLoader
# from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool
# from torch_geometric.utils import to_networkx

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import datasets

ROOT = os.getcwd().replace('ml/nlp', '')
print(ROOT)

dataset = datasets.load_dataset('ncbi_disease')
df_ncbi = pd.DataFrame(dataset['train'])

df_ncbi.head()

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/
Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 1519.86it/s]
Dataset ncbi_disease downloaded and prepared to /root/.cache/huggingface/datasets/ncbi_disease/ncbi_disease/1.0.0/92314c7992b0b8a5ea2ad101be33f365b684a2cc011e0ffa29c691e6d32b2d03. Subsequent calls will reuse this data.
100%|██████████| 3/3 [00:00<00:00, 177.51it/s]


,id,tokens,ner_tags
0,0,"[Identification, of, APC2, ,, a, homologue, of...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0]"
1,1,"[The, adenomatous, polyposis, coli, (, APC, ),...","[0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,"[Complex, formation, induces, the, rapid, degr...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,"[In, colon, carcinoma, cells, ,, loss, of, APC...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[Here, ,, we, report, the, identification, and...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [21]:
from torch.utils.data import Dataset
from torchtext.data.functional import custom_replace, simple_space_split
from itertools import chain
from nltk.corpus import stopwords

import nltk

!pip3 install nltk
nltk.download('stopwords')


def parse_xml(path):
    with open(path, 'r') as train_corpus: 
        def format_tag(line):
            all_starts = [idx for idx, item in enumerate(line) if '<passage>' in item]
            all_ends = [idx for idx, item in enumerate(line) if '</passage>' in item]
            return [[all_starts[idx], all_ends[idx]] for idx in range(len(all_starts))]
        data = train_corpus.readlines()

        ind = format_tag(data)
        return data, ind

def get_raw_txt(path):
    raw, indices = parse_xml(path)

    all_entries = [raw[indices[idx][0]+1:indices[idx][1]] for idx in range(len(indices))]
    all_txt = []

    for x, entry in enumerate(all_entries):
        empty = []
        for idx, item in enumerate(entry):
            if '<text>' in str(item):
                add_txt = entry[idx].replace('<text>', '')
                add_txt = add_txt.replace('</text>', '')
                add_txt = add_txt.replace('\n', '')
                empty.append(add_txt)
            else:
                pass
        all_txt.append(''.join(empty))
    return all_txt

custom_replace_transform = custom_replace([(r'\n', ''), (r'<', ''), (r'>', ''), (r'\t', '')])
preprocess = lambda x: (simple_space_split(custom_replace_transform(x)))

class PaperTextDataset(Dataset):
    def __init__ (self):
        self.data = {}
        self.embeddings = []
        self.length_axis = []
    
    def add_dataset(self, name, path, transformation):
        transformed_data = []
        for file in os.listdir(path):
            # try:
            processed_txt = list(preprocess(transformation(path+'/'+file)))
            processed_txt = list(chain.from_iterable(processed_txt))
            processed_txt = [x.lower() for x in processed_txt if (x not in stopwords.words('english')) or (x not in ['a', 'it', 'they', 'I', 'you', 'since', 'because', 'thus', 'therefore', 'however', 'some'])]
            transformed_data.append(processed_txt[:510])
            self.length_axis.append(len(processed_txt[:510]))
            # except:
                # print(path+file)
                # break

        # transformed_data = [preprocess(transformation(file)) for file in os.listdir(path)]
        self.data[name] = transformed_data
        return f"Added {name} to dataset."

    def __len__(self):
        return len(self.data)

    def __getitem__(self, name, idx):
        sample = self.data[name][idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def embed(self, tokenizer, model):
        self.embeddings = []
        for _, data in self.data.items():
            for text in data:
                with torch.no_grad():
                    inputs = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
                    outputs = model(inputs)
                    latent_output = outputs[0]
                    self.embeddings.append(latent_output)
        return self.embeddings

'/work/ml/data/nlm-chem-corpus/ALL/5525363_v1.xml'
db = PaperTextDataset()
db.add_dataset('nlm-chem-corpus', ROOT+'ml/data/nlm-chem-corpus/ALL/', get_raw_txt)

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'Added nlm-chem-corpus to dataset.'

In [22]:
import json
import onnx
from transformers import BertModel, BertConfig, BertTokenizer, BertForPreTraining, load_tf_weights_in_bert

#use biobert main because it's the parameters loaded into the BERT Cased
#reference for BERT mode: https://github.com/google-research/bert

CONFIG = 'ml/models/biobert_v1.1_pubmed/bert_config.json'
WEIGHTS = 'ml/models/biobert_v1.1_pubmed/model.ckpt-1000000.index'
LOAD_MODEL = 'ml/models/biobert.pt'

def main(config_path, init_checkpoint_path, model_save_path):
    print('Configuring BERT model parameters ...')
    config = BertConfig().from_json_file(ROOT+config_path)
    print('Loading BERT model weights ...')
    model = BertModel.from_pretrained(ROOT+WEIGHTS, config=config, from_tf=True)
    print('Saving BioBert Model ...')
    torch.save(model.state_dict(), ROOT+model_save_path)
    print('Done!')
    return model

biobert = main(CONFIG, WEIGHTS, LOAD_MODEL)
tokenizer = BertTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed', do_lower_case=True)
biobert.eval()

Configuring BERT model parameters ...
Loading BERT model weights ...
Saving BioBert Model ...
Done!


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [24]:
len(db.data['nlm-chem-corpus'][0])

510

In [25]:
import pickle
nlm_chem_corpus = db.embed(tokenizer=tokenizer, model=biobert)

with open(ROOT+'ml/models/embeddings_biobert', 'wb') as output_file:
    pickle.dump(nlm_chem_corpus, output_file)

with open(ROOT+"ml/models/embeddings_biobert", "rb") as input_file:
    nlm_embeddings = pickle.load(input_file)

In [27]:
print(nlm_chem_corpus[1].shape)

torch.Size([1, 512, 768])


In [28]:
# print(nlm_embeddings[0][0])
# print(nlm_chem_corpus[0][0])

array_nlp_embeddings = np.asarray([x.detach().numpy() for x in nlm_embeddings])
print(array_nlp_embeddings[0])

[[[-1.0685854e-03 -2.3804055e-01 -3.3589444e-01 ... -3.6289218e-01
   -1.9453727e-01  1.3101462e-01]
  [ 6.8597639e-01 -1.8458228e-01  2.0089817e-01 ... -3.0724707e-01
   -2.4389689e-01  3.9504102e-01]
  [ 1.1524965e+00 -3.2597321e-01 -1.1474037e-01 ... -1.2518139e-01
    4.6430829e-01  1.2401047e-02]
  ...
  [ 6.5270627e-01 -4.6341980e-01 -1.4489470e-01 ...  1.0597266e+00
   -4.7618228e-01 -1.1120352e-01]
  [ 5.8118361e-01 -4.4527462e-01 -1.0299656e+00 ... -4.0091872e-02
   -5.2802771e-01  1.0137630e-02]
  [ 1.8052539e-02  3.1354332e-01 -1.4269681e-01 ... -1.0099059e-01
   -8.4040093e-01 -4.3762574e-01]]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05362208-35d3-4fa0-ada7-4013d10444bf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>